In [ ]:
!pip install langchain-community langchain-chroma langchain-openai bs4

In [134]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv
import os

# [MY CODE] 환경변수 로드

In [135]:
load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

# [MY CODE] RAG internet source 설정 및 로드

In [136]:
loader = WebBaseLoader(
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("css-18vt64m")
        )
    ),
)
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content="스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆\xa0대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 

# [MY CODE] 스플리터

In [137]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200
)

splits = text_splitter.split_documents(docs)

# 문서 개수
print(len(splits))

# 전체 청크의 길이
total_chunk_size = sum(len(chunk.page_content) for chunk in splits)
print(f"전체 청크 사이즈: {total_chunk_size}")

3
전체 청크 사이즈: 5966


# [MY CODE] 임베딩 & 벡터 DB 저장

In [138]:
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY)
)

# [MY CODE] LLM 모델

In [139]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

# [MY CODE] 프롬프트 & 검색(Retrieval)

In [140]:
# 벡터스토어를 검색할 수 있는 리트리버 객체로 변환
retriever = vectorstore.as_retriever()

user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."

# 리트리버를 사용하여 사용자 메시지와 관련된 문서 검색
retrieved_docs = retriever.invoke(user_msg)

def format_docs(docs):
    # 각 문서의 내용을 연결하여 하나의 텍스트로 반환
    return "\n\n".join(doc.page_content for doc in docs)

prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
#print(user_prompt)


print("\n### 출처 문서 ###")
print(f'문서 개수: {len(retrieved_docs)}')
for doc in retrieved_docs:
    print(f"출처: {doc.metadata.get('source', 'N/A')}")
    print(f"내용: {doc.page_content[:200]}")  # 문서 내용 일부 출력

/opt/miniconda3/envs/hanghae99/lib/python3.10/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(



### 출처 문서 ###
문서 개수: 4
출처: https://spartacodingclub.kr/blog/all-in-challenge_winner
내용: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도
출처: https://spartacodingclub.kr/blog/all-in-challenge_winner
내용: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도
출처: https://spartacodingclub.kr/blog/all-in-challenge_winner
내용: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도
출처: https://spartacodingclub.kr/blog/all-in-challenge_winner
내용: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상

# [MY CODE] 질문 & 응답 & 토큰 수 확인

In [141]:
# OpenAI API 호출과 토큰 추적
with get_openai_callback() as callback:
    response = llm.invoke(user_prompt)

    # 토큰 관련 정보 출력
    print(f"입력 토큰 수: {callback.prompt_tokens}")
    print(f"응답 토큰 수: {callback.completion_tokens}")
    print(f"총 토큰 수: {callback.total_tokens}")
    print(f"총 비용: ${callback.total_cost}")

입력 토큰 수: 10450
응답 토큰 수: 110
총 토큰 수: 10560
총 비용: $0.0008654999999999998


In [142]:
print(response.content)

이번 'All-in 코딩 공모전'에서는 다양한 캠퍼스 문제를 해결하기 위한 혁신적인 프로젝트들이 발표되었습니다. 수상작 중 'Lexi Note'는 언어 학습을 효율적으로 도와주는 필기 웹 서비스이며, '우리집 히어로즈'는 벌레 퇴치를 위한 학생들 간의 매칭 서비스입니다. 또한 '에코 클래스룸'은 교수와 학생 간의 실시간 소통을 지원하여 학습 이해도를 높이는 서비스를 제공합니다.


# [MY CODE] RetrievalQA 사용

In [143]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                # 언어 모델
    chain_type="stuff",     # 검색된 모든 문서를 합쳐 전달 ("stuff" 방식)
    retriever=retriever,    # 벡터 스토어 리트리버
    return_source_documents=True  # 답변에 사용된 문서 출처 반환
)

In [144]:
# OpenAI API 호출과 토큰 추적
with get_openai_callback() as callback:
    response = qa_chain.invoke({"query": user_msg})

    # 토큰 관련 정보 출력
    print(f"입력 토큰 수: {callback.prompt_tokens}")
    print(f"응답 토큰 수: {callback.completion_tokens}")
    print(f"총 토큰 수: {callback.total_tokens}")
    print(f"총 비용: ${callback.total_cost}")

입력 토큰 수: 10437
응답 토큰 수: 480
총 토큰 수: 10917
총 비용: $0.0010855499999999998


In [145]:
print("\n### 출처 문서 ###")
print(f'문서 개수: {len(response["source_documents"])}')
for doc in response["source_documents"]:
    print(f"출처: {doc.metadata.get('source', 'N/A')}")
    print(f"내용: {doc.page_content[:200]}")  # 문서 내용 일부 출력


### 출처 문서 ###
문서 개수: 4
출처: https://spartacodingclub.kr/blog/all-in-challenge_winner
내용: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도
출처: https://spartacodingclub.kr/blog/all-in-challenge_winner
내용: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도
출처: https://spartacodingclub.kr/blog/all-in-challenge_winner
내용: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도
출처: https://spartacodingclub.kr/blog/all-in-challenge_winner
내용: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상

In [146]:
print(response["result"])

ALL-in 코딩 공모전에서 수상한 작품들은 다음과 같습니다:

1. **대상: [Lexi Note]** - 언어공부 필기 웹 서비스로, 단어를 드래그하면 네이버 사전과 연동되어 바로 의미를 찾고 필기를 할 수 있습니다. 또한, 파파고 번역기를 통해 긴 문장을 쉽게 이해할 수 있으며, 할일 목록과 스케줄 템플릿도 제공합니다. 사용 기술: React, Tesseract.js 등.

2. **우수상: [우리집 히어로즈]** - 대학생 자취생들을 위한 벌레 퇴치 영웅 매칭 서비스로, 벌레 문제를 해결하고자 하는 사용자가 요청을 올리면 같은 학교 학생과 매칭됩니다. 사용자 신원 보호 기능이 있습니다. 사용 기술: React Native, Spring Boot 등.

3. **우수상: [에코 클래스룸]** - 수업 중 학생들이 익명으로 의견이나 질문을 제출할 수 있는 실시간 소통 서비스입니다. 교수는 학생들의 이해도를 실시간으로 파악하고 피드백을 받을 수 있습니다. 사용 기술: Flutter, Spring Boot 등.

4. **입선: [Crewing]** - 대학생들이 연합 동아리를 쉽게 찾고 가입할 수 있도록 지원하는 정보 플랫폼입니다. 회원가입 시 개인 정보를 기반으로 적절한 동아리를 추천합니다. 사용 기술: Spring Boot, SwiftUI 등.

5. **입선: [학교생활 매니저]** - 대학 생활 관리 앱으로, 일정 및 과제 관리, 성적 예측 등의 기능을 제공합니다. 공지사항 확인 및 학점 계산기 기능도 포함되어 있습니다. 사용 기술: Flutter, Firebase 등.

6. **입선: [BLOTIE]** - 외국인 학생과 한국인 학생을 매칭하여 서로의 문화와 언어를 배우고 교류할 수 있는 플랫폼입니다. 관심사 및 언어 능력을 기반으로 매칭되며, 실시간 채팅 기능을 제공합니다. 사용 기술: React Native, Spring Boot 등.

이 작품들은 대학생들이 캠퍼스에서 겪는 다양한 문제를 해결하고자 하는 아이디어에서 출발하여 개발된 서비스들입

# rlm/rag-prompt VS Retriever

dfault 파라미터 에서는 검색에 큰 차이가 없음 오직 차이나는건 prompt
- rlm/rag-prompt : https://smith.langchain.com/hub/rlm/rag-prompt
- RetrieverQA : https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html

rlm/rag-prompt 프롬프트는 '최대 3개의 문장을 사용하고 답변을 간결하게 유지하세요.'라고 명시돼있다.
RetrieverQA는 마이그레이션 문서를 보면 system 역할을 사용하고 있는걸 알 수 있다. 실제로 프롬프트를 디버깅해보면 system에 담겨있다.

최대 3개의 문장을 사용하라고 했지만 질문을 하다보면 중요도에 따라 그 이상 주기도 한다.

![prompt_compare](./prompt_compare.png)

# [MY CODE] RetrieverQA의 시스템 프롬프트 이용

RetrieverQA은 0.1버전의 레거시이므로 마이그레이션 문서에 나와있는 RetrieverQA의 프롬프트를 직접 사용해보자.
(stuff, retriever 체인 사용)
(결과가 같다.)

In [199]:
import langchain
langchain.debug = False

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

#
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

response = chain.invoke({"input": user_msg})
print(response['answer'])

‘All-in 코딩 공모전’의 수상작은 다음과 같습니다:

1. **대상: Lexi Note** - 언어 공부를 위한 필기 웹 서비스로, 단어를 드래그하면 네이버 사전과 연동되고, 번역 버튼을 통해 번역기와 연동되어 학습 효율성을 높입니다.

2. **우수상: 우리집 히어로즈** - 대학생 자취생들을 위한 벌레 퇴치 서비스로, 사용자와 퇴치 히어로를 매칭하여 안전하게 문제를 해결할 수 있도록 지원합니다.

3. **우수상: 에코 클래스룸** - 교수와 학생 간의 실시간 소통을 위한 플랫폼으로, 익명으로 질문을 제출하고 이해도를 파악할 수 있는 기능을 제공합니다.

4. **입선: Crewing** - 연합 동아리 정보를 제공하는 플랫폼으로, 개인의 관심사에 맞는 동아리를 추천하고 가입 과정을 지원합니다.

5. **입선: 학교생활 매니저** - 대학생을 위한 관리 앱으로, 일정, 과제 관리 및 성적 예측 기능을 통해 학교 생활을 효율적으로 관리합니다.

6. **입선: BLOTIE** - 외국인 학생과 한국인 학생 간의 매칭 및 교류 플랫폼으로, 문화와 언어 교류를 촉진합니다.
